### 約定結果を確認するノートブック

In [1]:
import sys
sys.path.append('../')
import os

In [36]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
from datetime import datetime, timedelta
import ast

#自作モジュール
from gmo_api import GmoApi_private

### データの読み込み

In [42]:
data_dir = "../data/"

str_date = datetime(2023,8,23)
end_date = datetime(2023,8,24)

current_date = str_date

log_deal = pd.DataFrame()
log_execution = pd.DataFrame()

while current_date <= end_date:
    #log_dealの読み込み
    log_deal = pd.concat([log_deal, pd.read_csv(f"{data_dir}logs/log_deal/{current_date.strftime('%Y%m%d')}_deal_log.csv")],axis = 0)
    
    #log_executionの読み込み
    log_execution = pd.concat([log_execution, pd.read_csv(f"{data_dir}logs/log_execution/{current_date.strftime('%Y%m%d')}_execution_long.csv")], axis = 0)

    current_date += timedelta(days=1)


In [54]:
log_deal

,datetime,available_list,long_list,short_list
0,2023-08-23 01:25:10.891974,[],[],"['ASTR', 'BCH', 'BTC', 'ETH', 'XLM']"
1,datetime,available_list,long_list,short_list
2,2023-08-23 01:30:09.488117,[],[],['ETH']
3,datetime,available_list,long_list,short_list
4,2023-08-23 01:35:09.210749,[],[],['XRP']
...,...,...,...,...
562,2023-08-24 23:45:24.806480,"['XRP', 'XLM', 'XTZ', 'MKR', 'ASTR']",[],"['ASTR', 'BTC', 'ETH', 'LTC', 'XLM', 'XRP']"
563,datetime,available_list,long_list,short_list
564,2023-08-24 23:50:22.972224,"['XRP', 'XLM', 'XTZ', 'MKR', 'ASTR']",[],"['ASTR', 'BTC', 'ETH', 'LTC', 'XLM', 'XRP']"
565,datetime,available_list,long_list,short_list


In [55]:
log_execution

,executionId,fee,lossGain,orderId,price,settleType,side,size,symbol,timestamp
0,715142817,0,0,3773647507,9.755,OPEN,BUY,25.0000,ASTR,2023-08-23T19:46:20.035Z
1,714780824,4,0,3771585088,1074.000,OPEN,SELL,3.2100,ATOM,2023-08-23T02:01:59.673Z
2,714780825,1,0,3771585089,25.245,OPEN,SELL,30.0000,BAT,2023-08-23T02:02:00.959Z
3,714780826,4,0,3771585090,27311.000,OPEN,SELL,0.1600,BCH,2023-08-23T02:02:02.244Z
4,715143009,1,0,3773853730,3861705.000,OPEN,SELL,0.0002,BTC,2023-08-23T20:45:09.259Z
...,...,...,...,...,...,...,...,...,...,...
34,715411419,1,0,3775677048,75.899,OPEN,BUY,13.0000,XRP,2023-08-24T14:20:18.867Z
35,715328858,0,0,3775132903,76.245,OPEN,SELL,13.0000,XRP,2023-08-24T10:18:35.046Z
36,715328817,0,0,3775132659,76.368,OPEN,SELL,13.0000,XRP,2023-08-24T10:00:13.578Z
37,715328742,1,0,3775132422,76.396,OPEN,BUY,13.0000,XRP,2023-08-24T09:40:16.767Z


In [56]:
filtered_df = log_deal[log_deal['datetime'].str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d+$')]

In [57]:
filtered_df.loc[:,"datetime"] = pd.to_datetime(filtered_df["datetime"])
log_deal = filtered_df

In [58]:
log_deal

,datetime,available_list,long_list,short_list
0,2023-08-23 01:25:10.891974,[],[],"['ASTR', 'BCH', 'BTC', 'ETH', 'XLM']"
2,2023-08-23 01:30:09.488117,[],[],['ETH']
4,2023-08-23 01:35:09.210749,[],[],['XRP']
6,2023-08-23 02:05:16.096209,[],"['ASTR', 'ATOM', 'BAT', 'DOT', 'MONA']","['BCH', 'BTC', 'LINK', 'LTC', 'XEM', 'XRP']"
8,2023-08-23 02:10:12.554939,[],[],"['ADA', 'ETH', 'XRP']"
...,...,...,...,...
558,2023-08-24 23:35:21.417410,"['XRP', 'XLM', 'XTZ', 'MKR', 'ASTR']",[],"['ASTR', 'BTC', 'ETH', 'LTC', 'XLM', 'XRP']"
560,2023-08-24 23:40:23.231086,"['XRP', 'XLM', 'XTZ', 'MKR', 'ASTR']",[],"['ASTR', 'BTC', 'ETH', 'LTC', 'XLM', 'XRP']"
562,2023-08-24 23:45:24.806480,"['XRP', 'XLM', 'XTZ', 'MKR', 'ASTR']",[],"['ASTR', 'BTC', 'ETH', 'LTC', 'XLM', 'XRP']"
564,2023-08-24 23:50:22.972224,"['XRP', 'XLM', 'XTZ', 'MKR', 'ASTR']",[],"['ASTR', 'BTC', 'ETH', 'LTC', 'XLM', 'XRP']"
